# Job shop scheduling

<div class="alert alert-block alert-info">
    &#9432; The code in this notebook can be executed <a href="https://www.opvious.io/notebooks/retro/notebooks/?path=examples/job-shop-scheduling.ipynb">directly from your browser</a>.
</div>

In this notebook we implement the job shop scheduling problem described in https://jckantor.github.io/ND-Pyomo-Cookbook/notebooks/04.03-Job-Shop-Scheduling.html. We show in particular how [activation variable fragments](https://opvious.readthedocs.io/en/stable/api-reference.html#opvious.modeling.fragments.ActivationVariable) can be used to implement disjunctive constraints.

In [1]:
%pip install opvious

## Formulation

The first step is to formulate the problem problem using `opvious`' [declarative modeling API](https://opvious.readthedocs.io/en/stable/modeling.html).

In [2]:
import opvious.modeling as om

class JobShopScheduling(om.Model):
    """MIP formulation for scheduling dependent tasks of various durations"""
    
    tasks = om.Dimension()
    duration = om.Parameter.natural(tasks)
    machine = om.Parameter.natural(tasks)
    dependency = om.Parameter.indicator(tasks, tasks, qualifiers=['child', 'parent'])
    task_start = om.Variable.discrete(tasks, lower_bound=1)
    horizon = om.Variable.natural()
                                                  
    def task_end(self, t):
        return self.task_start(t) + self.duration(t)

    @om.constraint
    def all_tasks_end_within_horizon(self):
        for t in self.tasks:
            yield self.task_end(t) <= self.horizon()

    @om.constraint
    def child_starts_after_parent_ends(self):
        for c, p in self.tasks * self.tasks:
            if self.dependency(c, p):
                yield self.task_start(c) >= self.task_end(p)

    @property
    def competing_tasks(self):
        for t1, t2 in self.tasks * self.tasks:
            if t1 != t2 and self.machine(t1) == self.machine(t2):
                yield t1, t2

    @om.fragments.activation_variable(lambda init, self: init(self.competing_tasks, negate=True, upper_bound=self.duration.total()))
    def must_start_after(self, t1, t2):
        return self.task_end(t2) - self.task_start(t1)

    @om.fragments.activation_variable(lambda init, self: init(self.competing_tasks, negate=True, upper_bound=self.duration.total()))
    def must_end_before(self, t1, t2):
        return self.task_end(t1) - self.task_start(t2)

    @om.constraint
    def one_active_task_per_machine(self):
        for t1, t2 in self.competing_tasks:
            yield self.must_end_before(t1, t2) + self.must_start_after(t1, t2) >= 1

    @om.objective
    def minimize_horizon(self):
        return self.horizon()

model = JobShopScheduling()
model.specification()

<div style="margin-top: 1em; margin-bottom: 1em;">
<details open>
<summary style="cursor: pointer; text-decoration: underline; text-decoration-style: dotted;">JobShopScheduling</summary>
<div style="margin-top: 1em;">
$$
\begin{align*}
  \S^d_\mathrm{tasks}&: T \\
  \S^p_\mathrm{duration}&: d \in \mathbb{N}^{T} \\
  \S^p_\mathrm{machine}&: m \in \mathbb{N}^{T} \\
  \S^p_\mathrm{dependency[child,parent]}&: d' \in \{0, 1\}^{T \times T} \\
  \S^v_\mathrm{taskStart}&: \sigma^\mathrm{task} \in \{1 \ldots \infty\}^{T} \\
  \S^v_\mathrm{horizon}&: \eta \in \mathbb{N} \\
  \S^c_\mathrm{allTasksEndWithinHorizon}&: \forall t \in T, \sigma^\mathrm{task}_{t} + d_{t} \leq \eta \\
  \S^c_\mathrm{childStartsAfterParentEnds}&: \forall t, t' \in T \mid d'_{t,t'} \neq 0, \sigma^\mathrm{task}_{t} \geq \sigma^\mathrm{task}_{t'} + d_{t'} \\
  \S^v_\mathrm{mustStartAfter}&: \alpha^\mathrm{mustStart} \in \{0, 1\}^{\{ t, t' \in T \mid t \neq t' \land m_{t} = m_{t'} \}} \\
  \S^c_\mathrm{mustStartAfterActivates}&: \forall t, t' \in T \mid t \neq t' \land m_{t} = m_{t'}, \sum_{t'' \in T} d_{t''} \left(1 - \alpha^\mathrm{mustStart}_{t,t'}\right) \geq \sigma^\mathrm{task}_{t'} + d_{t'} - \sigma^\mathrm{task}_{t} \\
  \S^v_\mathrm{mustEndBefore}&: \beta^\mathrm{mustEnd} \in \{0, 1\}^{\{ t, t' \in T \mid t \neq t' \land m_{t} = m_{t'} \}} \\
  \S^c_\mathrm{mustEndBeforeActivates}&: \forall t, t' \in T \mid t \neq t' \land m_{t} = m_{t'}, \sum_{t'' \in T} d_{t''} \left(1 - \beta^\mathrm{mustEnd}_{t,t'}\right) \geq \sigma^\mathrm{task}_{t} + d_{t} - \sigma^\mathrm{task}_{t'} \\
  \S^c_\mathrm{oneActiveTaskPerMachine}&: \forall t, t' \in T \mid t \neq t' \land m_{t} = m_{t'}, \beta^\mathrm{mustEnd}_{t,t'} + \alpha^\mathrm{mustStart}_{t,t'} \geq 1 \\
  \S^o_\mathrm{minimizeHorizon}&: \min \eta \\
\end{align*}
$$
</div>
</details>
</div>

## Application

Let's try our formulation out on a simple example.

In [3]:
import opvious

async def find_optimal_start_times(tasks):
    """Returns a dataframe of optimal task start times"""
    problem = opvious.Problem(
        specification=model.specification(),
        parameters={
            'machine': {str(k): k[1] for k in tasks.keys()},
            'duration': {str(k): v['dur'] for k, v in tasks.items()},
            'dependency': [(str(k), str(v['prec'])) for k, v in tasks.items() if v['prec']]
        },
    )
    solution = await opvious.Client.default("https://try.opvious.io").solve(problem)
    return solution.outputs.variable('taskStart')

In [4]:
await find_optimal_start_times({
    ('Paper_1',1)   : {'dur': 45, 'prec': None},
    ('Paper_1',2) : {'dur': 10, 'prec': ('Paper_1',1)},
    ('Paper_2',1)   : {'dur': 20, 'prec': ('Paper_2',3)},
    ('Paper_2',3)  : {'dur': 10, 'prec': None},
    ('Paper_2',2) : {'dur': 34, 'prec': ('Paper_2',1)},
    ('Paper_3',1)   : {'dur': 12, 'prec': ('Paper_3',2)},
    ('Paper_3',3)  : {'dur': 17, 'prec': ('Paper_3',1)},
    ('Paper_3',2) : {'dur': 28, 'prec': None},   
})

,value
tasks,
"('Paper_1', 1)",43
"('Paper_1', 2)",88
"('Paper_2', 1)",11
"('Paper_2', 2)",31
"('Paper_2', 3)",1
"('Paper_3', 1)",31
"('Paper_3', 2)",1
"('Paper_3', 3)",43
